In [1]:
!pip install -q /kaggle/input/language-tool-python-2-7-1/language_tool_python-2.7.1-py3-none-any.whl

In [2]:
# %% Directory settings

# ====================================================
# Directory settings
# ====================================================
from pathlib import Path
import re

INPUT_DIR = Path("../input/")

In [3]:
import os
import zipfile
from zipfile import ZipFile
import shutil

# create download path
def get_language_tool_cache_path():
    # Get download path from environment or use default.
    download_path = os.environ.get(
        'LTP_PATH',
        os.path.join(os.path.expanduser("~"), ".cache", "language_tool_python")
    )
    # Make download path, if it doesn't exist.
    os.makedirs(download_path, exist_ok=True)
    return download_path

lt_path = get_language_tool_cache_path()
lt_path

'/root/.cache/language_tool_python'

In [4]:
#cant move files directly from input to cache, so we zip it to output and unzip again


def get_all_file_paths(directory):
  
    # initializing empty file paths list
    file_paths = []
  
    # crawling through directory and subdirectories
    for root, directories, files in os.walk(directory):
        for filename in files:
            # join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
  
    # returning all file paths
    return file_paths        
  
def main():
    # path to folder which needs to be zipped
    directory = '../input/language-tool-python-2-7-1/LanguageTool-5.7/LanguageTool-5.7'
  
    # calling function to get all file paths in the directory
    file_paths = get_all_file_paths(directory)

    # writing files to a zipfile
    with ZipFile('./lt.zip','w') as zip:
        # writing each file one by one
        for file in file_paths:
            zip.write(file)
  
    print('All files zipped successfully!')        
    
main()


 
zip_file = "./lt.zip"
 
try:
    with zipfile.ZipFile(zip_file) as z:
        z.extractall()
        print("Extracted all")
except:
    print("Invalid file")
    
#move to cache
!mv {'./input/language-tool-python-2-7-1/LanguageTool-5.7/LanguageTool-5.7'} {lt_path} 
print(os.listdir('/root/.cache/language_tool_python/'))

#remove files from output

shutil.rmtree('./input')
os.remove("./lt.zip")

All files zipped successfully!
Extracted all
['LanguageTool-5.7']


In [5]:
!pip install textstat -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.5 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset

ds = load_dataset("Jinyan1/COLING_2025_MGT_en")

README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/287M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/286M [00:00<?, ?B/s]

data/dev-00000-of-00001.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/610767 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/261758 [00:00<?, ? examples/s]

In [7]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'sub_source', 'lang', 'model', 'label', 'text'],
        num_rows: 610767
    })
    dev: Dataset({
        features: ['id', 'source', 'sub_source', 'lang', 'model', 'label', 'text'],
        num_rows: 261758
    })
})

In [8]:
df = ds['train'].to_pandas()[['text','label']].sample(5000,random_state=42)

In [9]:
df['label'].value_counts()

label
1    3109
0    1891
Name: count, dtype: int64

In [10]:
import polars as pl
import re
import string
from collections import Counter
from textblob import TextBlob
import textstat
import spacy
from scipy.stats import entropy
import gzip
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
import numpy as np
# from nltk.corpus import stopwords

# Load data
# train_df = pl.read_csv('/kaggle/input/pan-cief/train.csv')
# test_df = pl.read_csv('/kaggle/input/pan-cief/val.csv')

# Initialize spacy and stopwords
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

# Define all feature functions
def vocabSize(sentence):
    doc = nlp(sentence.lower())
    tokens = set([token.text for token in doc if not token.is_punct])
    return len(tokens)

def sentence_complexity(sentence):
    flesch_score = textstat.flesch_reading_ease(sentence)
    fk_grade_level = textstat.flesch_kincaid_grade(sentence)
    gunning_fog = textstat.gunning_fog(sentence)
    smog_index = textstat.smog_index(sentence)
    composite_score = (flesch_score * 0.2 + fk_grade_level * 0.3 + 
                      gunning_fog * 0.3 + smog_index * 0.2)
    return composite_score

def punctuation_count(paragraph):
    return sum(1 for char in paragraph if char in string.punctuation)

def sentence_length_difference(paragraph):
    sentences = re.split(r'[.!?]', paragraph)
    sentences = [s.strip() for s in sentences if s.strip()]
    if not sentences:
        return 0
    sentence_lengths = [len(s.split()) for s in sentences]
    return max(sentence_lengths) - min(sentence_lengths)

def type_token_ratio(text):
    words = text.split()
    if len(words) == 0:
        return 0
    unique_words = set(words)
    return len(unique_words) / len(words)

def pos_counts(text):
    doc = nlp(text)
    pos_count_dict = {}
    for token in doc:
        pos = token.pos_
        pos_count_dict[pos] = pos_count_dict.get(pos, 0) + 1
    return pos_count_dict

def count_discourse_markers(text):
    discourse_markers = ["however", "therefore", "although", "nevertheless", "hence"]
    return sum(text.lower().count(marker) for marker in discourse_markers)

def word_entropy(text):
    doc = nlp(text)
    words = [t.lemma_.lower() for t in doc if t.is_alpha]
    if not words:
        return 0
    freqs = list(Counter(words).values())
    return entropy(freqs)

def flesch_reading_ease(text):
    try:
        return textstat.flesch_reading_ease(text)
    except:
        return 0

def gzip_ratio(text):
    if len(text) == 0:
        return 0
    compressed = len(gzip.compress(text.encode('utf-8')))
    return compressed / len(text)

def negation_frequency(text):
    doc = nlp(text)
    negations = [t for t in doc if t.dep_ == "neg" or 
                 t.lemma_.lower() in ["not", "no", "never", "none", "n't"]]
    total_words = len([t for t in doc if t.is_alpha])
    return len(negations) / (total_words + 1e-5)

def question_statement_ratio(text):
    doc = nlp(text)
    sentences = list(doc.sents)
    if not sentences:
        return 0
    question_count = sum(1 for s in sentences if s.text.strip().endswith("?"))
    statement_count = sum(1 for s in sentences if s.text.strip().endswith("."))
    return question_count / (statement_count + 1e-5)

def clause_to_sentence_ratio(text):
    doc = nlp(text)
    sentences = list(doc.sents)
    if not sentences:
        return 0
    clause_markers = ("mark", "advcl", "ccomp", "xcomp", "acl", "relcl", "conj")
    clause_count = sum(1 for t in doc if t.dep_ in clause_markers)
    return clause_count / len(sentences)

def modal_verb_frequency(text):
    doc = nlp(text)
    modals = {"can", "could", "may", "might", "shall", "should", "will", "would", "must"}
    modal_count = sum(1 for t in doc if t.lemma_.lower() in modals)
    total_words = len([t for t in doc if t.is_alpha])
    return modal_count / (total_words + 1e-5)

def pronoun_ratio(text):
    doc = nlp(text)
    pronouns = [t for t in doc if t.pos_ == "PRON"]
    total_words = len([t for t in doc if t.is_alpha])
    return len(pronouns) / (total_words + 1e-5)

def pos_diversity(text):
    doc = nlp(text)
    pos_tags = [t.pos_ for t in doc if t.is_alpha]
    if not pos_tags:
        return 0
    counts = Counter(pos_tags)
    return entropy(list(counts.values()))

def hapax_legomena_ratio(text):
    doc = nlp(text)
    words = [t.lemma_.lower() for t in doc if t.is_alpha]
    if not words:
        return 0
    freqs = Counter(words)
    hapax = sum(1 for w, f in freqs.items() if f == 1)
    return hapax / len(freqs)

def get_sentiment_polarity(text):
    return TextBlob(text).sentiment.polarity

def get_sentiment_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def count_stopwords(text):
    return len([word for word in text.split() if word.lower() in stop_words])

In [11]:
from nltk.tokenize import sent_tokenize
from collections import Counter
import spacy
import re
from textblob import Word
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import math

nlp = spacy.load("en_core_web_sm")

def sentence_length_variation(text):
    sentences = sent_tokenize(text)
    lengths = [len(s.split()) for s in sentences if len(s.split()) > 0]

    if len(lengths) < 2:
        return 0.0  

    return np.std(lengths)   # Standard deviation

# def vocabulary_diversity(text): ## same as type token ratio
#     words = [w.lower() for w in text.split() if w.isalpha()]
#     if len(words) == 0:
#         return 0
#     return len(set(words)) / len(words)

def repetition_rate(text):
    words = [w.lower() for w in text.split()]
    bigrams = [(words[i], words[i+1]) for i in range(len(words)-1)]
    if len(bigrams) == 0:
        return 0

    counts = Counter(bigrams)
    repeated = sum(1 for bg, c in counts.items() if c > 1)

    return repeated / len(bigrams)

def personal_voice_score(text):
    personal_pronouns = {"i", "me", "my", "mine", "we", "us", "our", "ours"}
    words = [w.lower() for w in text.split()]
    count = sum(1 for w in words if w in personal_pronouns)
    if len(words) == 0:
        return 0
    return count / len(words)

def emotion_variation(text):
    sentences = sent_tokenize(text)
    if len(sentences) < 2:
        return 0

    sentiments = [TextBlob(s).sentiment.polarity for s in sentences]
    diffs = [abs(sentiments[i] - sentiments[i+1]) for i in range(len(sentiments)-1)]

    return np.mean(diffs)


def specificity_score(text):
    doc = nlp(text)
    concrete_tags = {"NOUN", "PROPN", "NUM"}  
    concrete_count = sum(1 for token in doc if token.pos_ in concrete_tags)
    if len(doc) == 0:
        return 0
    return concrete_count / len(doc)


def imperfection_score(text):
    words = [w for w in re.findall(r"\b\w+\b", text)]
    if len(words) == 0:
        return 0

    misspelled = sum(1 for w in words if Word(w).correct().lower() != w.lower())
    return misspelled / len(words)

figurative_markers = [
    "like", "as if", "as though", "metaphor", "symbolic", 
    "resembles", "reminds me of", "figurative"
]

def figurative_language_score(text):
    t = text.lower()
    count = sum(t.count(m) for m in figurative_markers)
    return count

def paragraph_coherence_consistency(text):
    paragraphs = [p.strip() for p in text.split("\n") if len(p.strip()) > 0]

    if len(paragraphs) < 2:
        return 0

    vec = TfidfVectorizer().fit_transform(paragraphs)
    sims = []

    for i in range(len(paragraphs)-1):
        sim = cosine_similarity(vec[i], vec[i+1])[0][0]
        sims.append(sim)

    return np.mean(sims)


def predictability_score(text):
    words = [w.lower() for w in text.split()]
    counts = Counter(words)
    total = len(words)
    if total == 0:
        return 0

    probs = [counts[w]/total for w in words]
    surprise = [-math.log(p) for p in probs]

    return np.mean(surprise)

hedge_words = {
    "maybe", "perhaps", "sort of", "kind of", "i guess", 
    "probably", "possibly", "apparently", "roughly"
}

def hedge_uncertainty_score(text):
    t = text.lower()
    count = sum(t.count(hw) for hw in hedge_words)
    return count

transitions = [
    "however", "therefore", "meanwhile", "moreover", "furthermore",
    "in contrast", "on the other hand", "overall", "in summary"
]

def transition_variety_score(text):
    t = text.lower()
    count = sum(t.count(word) for word in transitions)
    return count

In [12]:
# from nltk.tokenize import sent_tokenize
# from collections import Counter
# import spacy
# import re
# from textblob import Word
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from collections import Counter
# import math
import language_tool_python

tool = language_tool_python.LanguageTool('en-US')
# nlp = spacy.load("en_core_web_sm")

In [13]:
def grammatical_mistakes(sentence):

  mistakes = len(tool.check(sentence))

  return mistakes

In [14]:
def pos_tag_ngrams(text, n=2):
    doc = nlp(text)
    tags = [token.pos_ for token in doc if token.is_alpha]

    if len(tags) < n:
        return {}

    ngrams = zip(*[tags[i:] for i in range(n)])
    return Counter(ngrams)

def pos_ngram_variety(text, n=2):
    ngrams = pos_tag_ngrams(text, n)
    return len(ngrams)

In [15]:
# import torch
# from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# # Load once (place globally)
# tokenizer = GPT2TokenizerFast.from_pretrained("/kaggle/input/gpt2/transformers/gpt21/1")
# gpt_model = GPT2LMHeadModel.from_pretrained("/kaggle/input/gpt2/transformers/gpt21/1")
# gpt_model.eval()

# def perplexity(text):
#     enc = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024)
#     input_ids = enc.input_ids

#     with torch.no_grad():
#         outputs = gpt_model(input_ids, labels=input_ids)

#     loss = outputs.loss
#     return torch.exp(loss).item()

In [16]:
import pandas as pd
from textblob import TextBlob

def get_features_from_text(text: str):
    stop_words = set(stopwords.words('english'))
    
    features = {}
    features['character_count'] = len(text)
    features['word_count'] = len(text.split())
    features['sentence_count']=len(sent_tokenize(text))
    features['paragraph_count'] = len(text.split("\n"))
    features['stopword_count'] = len([word for word in text.split() if word.lower() in stop_words])
    features['unique_word_count'] = len(set(text.split()))
    features['sentiment_polarity'] = TextBlob(text).sentiment.polarity
    features['sentiment_subjectivity'] = TextBlob(text).sentiment.subjectivity
    features['discourse_marker_count'] = count_discourse_markers(text)
    features['vocab_size'] = vocabSize(text)
    features['sentence_complexity'] = sentence_complexity(text)
    features['punctuation_count'] = punctuation_count(text)
    features['sentence_length_difference'] = sentence_length_difference(text)
    features['type_token_ratio'] = type_token_ratio(text)
    features['word_entropy'] = word_entropy(text)
    features['flesch_reading_ease'] = flesch_reading_ease(text)
    features['gzip_ratio'] = gzip_ratio(text)
    features['negation_freq'] = negation_frequency(text)
    features['question_stmt_ratio'] = question_statement_ratio(text)
    features['clause_sentence_ratio'] = clause_to_sentence_ratio(text)
    features['modal_freq'] = modal_verb_frequency(text)
    features['pronoun_ratio'] = pronoun_ratio(text)
    features['pos_diversity'] = pos_diversity(text)
    features['hapax_ratio'] = hapax_legomena_ratio(text)
    features['sentence_length_variation'] = sentence_length_variation(text)
    features['repetition_rate'] = repetition_rate(text)
    features['personal_voice_score'] = personal_voice_score(text)
    features['emotion_variation'] = emotion_variation(text)
    features['specificity_score'] = specificity_score(text)
    features['figurative_language_score'] = figurative_language_score(text)
    features['paragraph_coherence_consistency'] = paragraph_coherence_consistency(text)
    features['predictability_score'] = predictability_score(text)
    features['hedge_uncertainty_score'] = hedge_uncertainty_score(text)
    features['transition_variety_score'] = transition_variety_score(text)
    features['grammatical_mistakes'] = grammatical_mistakes(text)
    features['pos_2gram_variety'] = pos_ngram_variety(text)
    features['pos_3gram_variety'] = pos_ngram_variety(text,n=3)
    features['pos_4gram_variety'] = pos_ngram_variety(text,n=4)
    # features['perplexity'] = perplexity(text)
    
    return pd.Series(features)

# Apply the function to create new columns
new_features = df['text'].apply(get_features_from_text)
df = pd.concat([df, new_features], axis=1)

In [17]:
df.to_csv('coling_train.csv')